In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
import xgboost as xgb

# Load the dataset
data = pd.read_csv("new_dataset.csv")  # Replace with the actual dataset file

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Extract year and month as features
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Define the target variables
target_rainfall = 'Rainfall_mm'
target_temperature = 'Temperature_Celsius'
target_weather = 'Weather'
target_energy_demand = 'Energy Demand (GWh)'

# Split the dataset into training and test sets
train_data = data[data['Year'] < 2023]
test_data = data[(data['Year'] == 2023) & (data['Month'] == 9)]

# Features for prediction
features = ['Year', 'Month', 'Solar Energy (GWh)', 'Hydroelectric Power (GWh)']

# Predict Rainfall
X = train_data[features]
y = train_data[target_rainfall]
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X, y)
predicted_rainfall = rf_regressor.predict(test_data[features])

# Predict Temperature
X = train_data[features]
y = train_data[target_temperature]
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X, y)
predicted_temperature = rf_regressor.predict(test_data[features])

# Predict Weather (Rainy/Sunny)
X = train_data[features]
y = train_data[target_weather]
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X, y)
predicted_weather = rf_classifier.predict(test_data[features])

# Predict Energy Demand
X = train_data[features]
y = train_data[target_energy_demand]
xgb_regressor = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_regressor.fit(X, y)
predicted_energy_demand = xgb_regressor.predict(test_data[features])

# Suggest Power Generation Source
suggested_power_source = ['Solar' if weather == 'Sunny' else 'Hydro' for weather in predicted_weather]

# Calculate the energy deficit or surplus
predicted_energy_generation = []  # This list will store the predicted energy generation for each month
for i in range(len(test_data)):
    # Calculate the energy deficit or surplus for each month
    energy_deficit_surplus = predicted_energy_demand[i] - (predicted_rainfall[i] + predicted_temperature[i])

    # Adjust energy generation based on the power source
    if suggested_power_source[i] == 'Solar':
        predicted_energy_generation.append(energy_deficit_surplus)  # Solar can directly meet the demand
    else:
        predicted_energy_generation.append(energy_deficit_surplus)

# Create a DataFrame for predictions
predictions = pd.DataFrame({
    'Predicted_Rainfall_mm': predicted_rainfall,
    'Predicted_Temperature_Celsius': predicted_temperature,
    'Predicted_Weather': predicted_weather,
    'Predicted_Energy_Demand_GWh': predicted_energy_demand,
    'Suggested_Power_Source': suggested_power_source,
    'Predicted_Energy_Generation_GWh': predicted_energy_generation
})

# Initialize lists to store power generation from both sources
predicted_solar_energy_generation = []
predicted_hydro_energy_generation = []

for i in range(len(test_data)):
    # Calculate the energy deficit or surplus for each month
    energy_deficit_surplus_i = energy_deficit_surplus

    # Initialize power generation from both sources
    solar_energy_generation = 0
    hydro_energy_generation = 0

    # Calculate solar energy generation if it's the suggested power source
    if suggested_power_source[i] == 'Solar':
        solar_energy_generation = min(energy_deficit_surplus_i, predicted_energy_demand[i])
        energy_deficit_surplus_i -= solar_energy_generation
    
    # Calculate hydro energy generation if it's the suggested power source
    if suggested_power_source[i] == 'Hydro':
        hydro_energy_generation = min(energy_deficit_surplus_i, predicted_energy_demand[i])
    
    predicted_solar_energy_generation.append(solar_energy_generation)
    predicted_hydro_energy_generation.append(hydro_energy_generation)

# Create a DataFrame for predictions
predictions = pd.DataFrame({
    'Predicted_Rainfall_mm': predicted_rainfall,
    'Predicted_Temperature_Celsius': predicted_temperature,
    'Predicted_Weather': predicted_weather,
    'Predicted_Energy_Demand_GWh': predicted_energy_demand,
    'Suggested_Power_Source': suggested_power_source,
    'Predicted_Solar_Energy_GWh': predicted_solar_energy_generation,
    'Predicted_Hydro_Energy_GWh': predicted_hydro_energy_generation
})

# Display the predictions
print(predictions)


   Predicted_Rainfall_mm  Predicted_Temperature_Celsius Predicted_Weather  \
0             110.055178                      25.918277             Sunny   

   Predicted_Energy_Demand_GWh Suggested_Power_Source  \
0                  4649.504395                  Solar   

   Predicted_Solar_Energy_GWh  Predicted_Hydro_Energy_GWh  
0                 4513.530939                           0  


In [10]:
test_data

,Date,Energy Demand (GWh),Solar Energy (GWh),Hydroelectric Power (GWh),Temperature_Celsius,Rainfall_mm,Weather,Year,Month
165,2023-10-31,0,0.0,0,0.0,0.0,0,2023,10


In [17]:
import requests

# Replace 'YOUR_API_KEY' with your actual API key
api_key = 'b0ff80d841cb45b28e8194148231409'
weather_api_url = f'http://api.weatherapi.com/v1/current.json?key=b0ff80d841cb45b28e8194148231409&q=idukki&aqi=no'

# Make an API request
response = requests.get(weather_api_url)

# Parse the JSON response to extract weather data
weather_data = response.json()


In [18]:
# Make an HTTP GET request to the API
response = requests.get(weather_api_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    weather_data = response.json()

    # Extract temperature in Celsius and rainfall in mm from the JSON response
    temperature_celsius = weather_data['current']['temp_c']
    rainfall_mm = weather_data['current']['precip_mm']

    # Now you can use temperature_celsius and rainfall_mm in your AI model

    # Print the extracted values
    print(f"Temperature (Celsius): {temperature_celsius}")
    print(f"Rainfall (mm): {rainfall_mm}")
else:
    print(f"Failed to fetch data. Status Code: {response.status_code}")

Temperature (Celsius): 18.9
Rainfall (mm): 0.1


In [5]:
test_data

,Date,Energy Demand (GWh),Solar Energy (GWh),Hydroelectric Power (GWh),Temperature_Celsius,Rainfall_mm,Weather,Year,Month
156,2023-01-31,4698,71.476035,4712,30.182974,23.3,Sunny,2023,1


In [15]:
import pandas as pd
import numpy as np

# Load the original dataset
data = pd.read_csv("kerala_energy_data.csv")  # Replace with the actual dataset file

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%Y')

# Extract the month as a feature
data['Month'] = data['Date'].dt.month

# Create a new dataset to store the split power values
new_data = pd.DataFrame()

# Define a function to split the power based on season
def split_power(row):
    if row['Weather'] == 'Sunny':
        # Allocate more power to solar during summer
        solar_power = row['Total_maxpower (gwh)'] * 0.7
        hydro_power = row['Total_maxpower (gwh)'] * 0.3
    else:
        # Allocate more power to hydro during rainy season
        solar_power = row['Total_maxpower (gwh)'] * 0.3
        hydro_power = row['Total_maxpower (gwh)'] * 0.7
    
    row['Solar Energy (GWh)'] = solar_power
    row['Hydroelectric Power (GWh)'] = hydro_power
    return row

# Apply the split_power function to each row in the original dataset
new_data = data.apply(split_power, axis=1)

# Save the new dataset to a CSV file
new_data.to_csv("new_dataset.csv", index=False)


In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost as xgb

# Load the dataset
data = pd.read_csv("new_dataset.csv", parse_dates=["Date"])

# Extract year and month as features
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Define the target variables
target_weather = 'Weather'
target_energy_demand = 'Energy Demand (GWh)'

# Features for prediction
features = ['Year', 'Month', 'Solar Energy (GWh)', 'Hydroelectric Power (GWh)','Energy Demand (GWh)','Total_maxpower (gwh)']

# Split the dataset into training data
train_data = data[data['Year'] < 2023]

# Predict Weather (Rainy/Sunny)
X = train_data[features]
y = train_data[target_weather]
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X, y)

# Predict Energy Demand
X = train_data[features]
y = train_data[target_energy_demand]
xgb_regressor = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_regressor.fit(X, y)

# Function to predict for user input date, temperature, and rainfall
def predict_for_user_input(input_date, input_temperature, input_rainfall):
    # Extract year and month from the input date
    input_year = input_date.year
    input_month = input_date.month
    
    # Find the most recent available data for the same month and year
    recent_data = train_data[(train_data['Year'] == input_year) & (train_data['Month'] == input_month)]
    
    if recent_data.empty:
        # If there's no data for the specified month and year, use the most recent data for the same month
        recent_data = train_data[train_data['Month'] == input_month].tail(1)
    
    input_features = recent_data[features].values.reshape(1, -1)
    
    # Predict Energy Demand
    predicted_energy_demand = xgb_regressor.predict(input_features)[0]
    
    # Suggest Power Generation Source
    suggested_power_source = 'Solar' if input_temperature > 25 and input_rainfall < 50 else 'Hydro'
    
    # Predict Weather
    predicted_weather = rf_classifier.predict(input_features)[0] if input_temperature and input_rainfall is None else 'Rainy' if input_rainfall >= 50 else 'Sunny'
    
    # Calculate Total Power Needed to be Produced (more than demand)
    total_power_needed = predicted_energy_demand * 1.1  
    
    # Calculate Power to be Produced by Solar and Hydro
    if suggested_power_source == 'Solar':
        power_hydro = total_power_needed / 5
        power_solar = total_power_needed - power_hydro
    else:
        power_solar = total_power_needed / 5
        power_hydro = total_power_needed - power_solar
    
    return {
        'Predicted_Weather': predicted_weather,
        'Predicted_Energy_Demand_GWh': predicted_energy_demand,
        'Suggested_Power_Source': suggested_power_source,
        'Total_Power_Needed_GWh': total_power_needed,
        'Power_Produced_by_Solar_GWh': power_solar,
        'Power_Produced_by_Hydro_GWh': power_hydro
    }

# Get user input for the date, temperature, and rainfall
user_input_date_str = input("Enter a date (YYYY-MM): ")
user_input_date = pd.to_datetime(user_input_date_str, format='%Y-%m')

user_input_temperature = float(input("Enter temperature (Celsius): "))
user_input_rainfall = float(input("Enter rainfall (mm): "))

# Predict for user input date, temperature, and rainfall
prediction = predict_for_user_input(user_input_date, user_input_temperature, user_input_rainfall)

# Display the prediction
print("\nPrediction for Date:", user_input_date.strftime('%Y-%m'))
print("Predicted Weather:", prediction['Predicted_Weather'])
print("Predicted Energy Demand (MWh):", prediction['Predicted_Energy_Demand_GWh'])
print("Suggested Power Source:", prediction['Suggested_Power_Source'])
print("Total Power Needed to be Produced (MWh):", prediction['Total_Power_Needed_GWh'])
print("Power Produced by Solar (MWh):", prediction['Power_Produced_by_Solar_GWh'])
print("Power Produced by Hydro (MWh):", prediction['Power_Produced_by_Hydro_GWh'])


Enter a date (YYYY-MM): 2023-10
Enter temperature (Celsius): 23
Enter rainfall (mm): 678

Prediction for Date: 2023-10
Predicted Weather: Rainy
Predicted Energy Demand (GWh): 4612.0244
Suggested Power Source: Hydro
Total Power Needed to be Produced (GWh): 5073.226855468751
Power Produced by Solar (GWh): 1014.6453710937501
Power Produced by Hydro (GWh): 4058.5814843750004


In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost as xgb

# Load the dataset
data = pd.read_csv("new_dataset.csv", parse_dates=["Date"])

# Extract year and month as features
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Define the target variables
target_rainfall = 'Rainfall_mm'
target_temperature = 'Temperature_Celsius'
target_weather = 'Weather'
target_energy_demand = 'Energy Demand (GWh)'

# Features for prediction
features = ['Year', 'Month', 'Solar Energy (GWh)', 'Hydroelectric Power (GWh)','Energy Demand (GWh)','Total_maxpower (gwh)']

# Split the dataset into training data
train_data = data[data['Year'] < 2023]

# Predict Weather (Rainy/Sunny)
X = train_data[features]
y = train_data[target_weather]
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X, y)

# Predict Energy Demand
X = train_data[features]
y = train_data[target_energy_demand]
xgb_regressor = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_regressor.fit(X, y)

# Predict Temperature
X = train_data[features]
y = train_data[target_temperature]
temperature_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
temperature_regressor.fit(X, y)

# Predict Rainfall
X = train_data[features]
y = train_data[target_rainfall]
rainfall_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rainfall_regressor.fit(X, y)

# Function to predict for user input date
def predict_for_user_input(input_date):
    # Extract year and month from the input date
    input_year = input_date.year
    input_month = input_date.month
    
    # Use the previous year's data to predict
    previous_year_data = train_data[(train_data['Year'] == (input_year - 1)) & (train_data['Month'] == input_month)]
    if previous_year_data.empty:
        # If there's no data for the specified month and year, use the most recent data for the same month
        previous_year_data = train_data[train_data['Month'] == input_month].tail(1)

    input_features = previous_year_data[features].values.reshape(1, -1)
    
    # Predict Weather
    predicted_weather = rf_classifier.predict(input_features)[0]
    
    # Predict Energy Demand
    predicted_energy_demand = xgb_regressor.predict(input_features)[0]
    
    # Predict Temperature
    predicted_temperature = temperature_regressor.predict(input_features)[0]
    
    # Predict Rainfall
    predicted_rainfall = rainfall_regressor.predict(input_features)[0]
    
    # Suggest Power Generation Source
    suggested_power_source = 'Solar' if predicted_weather == 'Sunny' else 'Hydro'
    
    # Calculate Total Power Needed to be Produced (more than demand)
    total_power_needed = predicted_energy_demand * 1.1  
    
    # Calculate Power to be Produced by Solar and Hydro
    if suggested_power_source == 'Solar':
        power_hydro = total_power_needed / 5
        power_solar = total_power_needed - power_hydro
    else:
        power_solar = total_power_needed / 5
        power_hydro = total_power_needed - power_solar
    
    return {
        'Predicted_Weather': predicted_weather,
        'Predicted_Energy_Demand_GWh': predicted_energy_demand,
        'Suggested_Power_Source': suggested_power_source,
        'Total_Power_Needed_GWh': total_power_needed,
        'Power_Produced_by_Solar_GWh': power_solar,
        'Power_Produced_by_Hydro_GWh': power_hydro,
        'Predicted_Temperature_Celsius': predicted_temperature,
        'Predicted_Rainfall_mm': predicted_rainfall
    }

# Get user input for the date
user_input_date_str = input("Enter a date (YYYY-MM): ")
user_input_date = pd.to_datetime(user_input_date_str, format='%Y-%m')

# Predict for user input date
prediction = predict_for_user_input(user_input_date)

# Display the prediction
print("\nPrediction for Date:", user_input_date.strftime('%Y-%m'))
print("Predicted Weather:", prediction['Predicted_Weather'])
print("Predicted Energy Demand (GWh):", prediction['Predicted_Energy_Demand_GWh'])
print("Suggested Power Source:", prediction['Suggested_Power_Source'])
print("Total Power Needed to be Produced (GWh):", prediction['Total_Power_Needed_GWh'])
print("Power Produced by Solar (GWh):", prediction['Power_Produced_by_Solar_GWh'])
print("Power Produced by Hydro (GWh):", prediction['Power_Produced_by_Hydro_GWh'])
print("Predicted Temperature (Celsius):", prediction['Predicted_Temperature_Celsius'])
print("Predicted Rainfall (mm):", prediction['Predicted_Rainfall_mm'])


Enter a date (YYYY-MM): 2023-04

Prediction for Date: 2023-04
Predicted Weather: Sunny
Predicted Energy Demand (GWh): 4548.9517
Suggested Power Source: Solar
Total Power Needed to be Produced (GWh): 5003.846826171875
Power Produced by Solar (GWh): 4003.0774609375003
Power Produced by Hydro (GWh): 1000.7693652343751
Predicted Temperature (Celsius): 34.04999999999999
Predicted Rainfall (mm): 103.59900000000002
